In [3]:
# !pip install folium branca
# !pip install -U pystac-client

In [4]:
import branca
from folium import Map, TileLayer
import json
from matplotlib import cm
import requests

# Discover data from STAC

In [17]:
stac_endpoint = "https://stac.maap-project.org"
titiler_endpoint = "https://titiler.maap-project.org/"
collection = "NASA_JPL_global_agb_mean_2020"
item = 'NAmerica'
items_response = requests.get(f"{stac_endpoint}/collections/{collection}/items/{item}").json()
url = items_response['assets']['mean']['href']
url

's3://nasa-maap-data-store/file-staging/nasa-map/NASA_JPL_global_agb_mean_2020/global_008_06dc_agb_mean_prediction_2020_mosaic_veg_gfccorr_scale1_NAmerica_cog.tif'

# Get data values from the `/statistics` endpoint

In [18]:
# You can use gdalinfo /vsis3/nasa-maap-data-store/file-staging/nasa-map/SRTMGL1_COD___001/N21W089.SRTMGL1.tif -stats
# or you can get metadata from titiler.
stats_response = requests.get(
    f"{titiler_endpoint}/cog/statistics",
    params = {
        "url": url
    }
).json()

In [19]:
bins = stats_response['b1']['histogram'][1]
bin_ranges = [[bins[i], bins[i+1]] for i in range(len(bins)-1)]
bin_ranges

[[-148.0, -93.2],
 [-93.2, -38.400000000000006],
 [-38.400000000000006, 16.399999999999977],
 [16.399999999999977, 71.19999999999999],
 [71.19999999999999, 126.0],
 [126.0, 180.79999999999995],
 [180.79999999999995, 235.59999999999997],
 [235.59999999999997, 290.4],
 [290.4, 345.2],
 [345.2, 400.0]]

In [20]:
# There are many pre-defined colormaps supported by matplotlib.
# Some are listed below but a complete list be found here: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
# You may define custom color maps, but using the predefined ones makes life easier.
sequential_cmaps = [
    'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
    'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
    'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn'
]

sequential_cmaps2 = [
    'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
    'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
    'hot', 'afmhot', 'gist_heat', 'copper']

In [21]:
# Here we create a list of pairs, each pair containing a data value interval range (aka "bin")
# with a color value in RGBA (see https://www.w3schools.com/css/css3_colors.asp)
# First we create a list of colors
rgbas = [[int(value) for value in rgb] for rgb in cm.ScalarMappable(cmap='gist_earth').to_rgba(x=bins[:-1], bytes=True)]
# Then we use python's zip function to pair rgba values with data values (https://www.w3schools.com/python/ref_func_zip.asp)
color_map = list(zip(bin_ranges, rgbas))
# some tweaking may be necessary

In [22]:
# Let's also create a legend using the RGBA values and bins so our map visualization can be interpreted!
legend = branca.colormap.StepColormap(rgbas, index=bins, vmin=round(bins[0], 2), vmax=round(bins[-1], 2))

# Preview the data

In [23]:
# Create a json string of the colormap, so it can be passed as a parameter to titiler's API.
cmap = json.dumps(color_map)

# We fetch tilejson from titiler endpoint, to build a better map with appropriate bounds and zoom level
tilejson_response = requests.get(
    f"{titiler_endpoint}/cog/tilejson.json",
    params = {
        "url": url,
        "colormap": cmap
    }
).json()
tilejson_response
bounds = tilejson_response["bounds"]
m = Map(
    location=((bounds[1] + bounds[3]) / 2,(bounds[0] + bounds[2]) / 2),
    zoom_start=tilejson_response["minzoom"] + 1
)

# We add a TileLayer using the tilejson_response "tiles" value which is the XYZ endpoint of titiler.
aod_layer = TileLayer(
    tiles=tilejson_response["tiles"][0],
    opacity=1,
    attr="Swisstopo"
)
aod_layer.add_to(m)

# Finally, we add the legend.
legend.caption = 'Estimated aboveground biomass in Mg/ha (megagram per hectare)'
legend.add_to(m)

m